In [1]:
import os
import gc
import math

import numpy as np
import miditok
from miditok import REMI, TokenizerConfig
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from symusic import Score

from pathlib import Path

In [2]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = "0"

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import Optimizer
from torch.nn import CrossEntropyLoss
import torchaudio

In [4]:
from transformers import SEWModel, AutoProcessor

2024-06-06 22:16:31.159800: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 22:16:31.187197: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 22:16:31.809371: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
processor = AutoProcessor.from_pretrained("asapp/sew-tiny-100k-ft-ls100h")
model = SEWModel.from_pretrained("asapp/sew-tiny-100k-ft-ls100h")

/home/bk/miniconda3/envs/keras/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/163M [00:00<?, ?B/s]

/home/bk/miniconda3/envs/keras/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [6]:
model.cuda()

SEWModel(
  (feature_extractor): SEWFeatureEncoder(
    (conv_layers): ModuleList(
      (0): SEWGroupNormConvLayer(
        (conv): Conv1d(1, 64, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(64, 64, eps=1e-05, affine=True)
      )
      (1): SEWNoLayerNormConvLayer(
        (conv): Conv1d(64, 128, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (2): SEWNoLayerNormConvLayer(
        (conv): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
        (activation): GELUActivation()
      )
      (3): SEWNoLayerNormConvLayer(
        (conv): Conv1d(128, 128, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (4): SEWNoLayerNormConvLayer(
        (conv): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
        (activation): GELUActivation()
      )
      (5): SEWNoLayerNormConvLayer(
        (conv): Conv1d(128,

In [12]:
audio, sr = torchaudio.load("song2midi/dataset/song/anima__full.mp3")
audio = torch.mean(audio, dim=0)
audio = torchaudio.transforms.Resample(sr, 16000)(audio)

In [13]:
inputs = processor(audio, sampling_rate=16000, return_tensors="pt")

In [14]:
inputs["input_values"].shape

torch.Size([1, 4274838])

In [17]:
with torch.no_grad():
    outputs = model(inputs["input_values"].cuda())

/home/bk/miniconda3/envs/keras/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


In [19]:
outputs.last_hidden_state.shape

torch.Size([1, 13358, 512])

In [19]:
encoder_outputs = encodec_model.encode(inputs["input_values"].cuda(), inputs["padding_mask"].cuda())

/home/bk/miniconda3/envs/keras/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


In [22]:
encoder_outputs.audio_codes.shape

torch.Size([1, 1, 2, 20039])

In [8]:
from transformers import AutoTokenizer, MambaConfig, MambaForCausalLM

![dataset](images/Dataset.png)

In [5]:
a = [name.split(".")[0] for name in os.listdir("song2midi/dataset/song")]

In [11]:
b = [name.split(".")[:2] for name in os.listdir("song2midi/dataset/midi")]

In [12]:
[midi for midi in b if midi[0] in a]

[['1', '1'],
 ['10', '1'],
 ['2', '1'],
 ['3', '1'],
 ['4', '1'],
 ['5', '1'],
 ['6', '1'],
 ['7', '1'],
 ['8', '1'],
 ['9', '1']]

In [7]:
torch.cat([torch.rand((1, 128, 1024)), torch.rand((1, 1, 1024))], dim=-2).shape

torch.Size([1, 129, 1024])

In [14]:
(torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) + torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))

tensor([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20])

In [9]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")

/home/bk/miniconda3/envs/keras/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
tokenizer("Hello, my dog is cute", return_tensors="pt")

{'input_ids': tensor([[12092,    13,   619,  4370,   310, 20295]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [22]:
class MambaEmbedding(nn.Module):
  def __init__(self, vocab_size, arranger_ids=256, hidden_size=128):
    super().__init__()
    self.arranger_embeddings = nn.Embedding(arranger_ids, hidden_size)
    self.token_embeddings = nn.Embedding(vocab_size, hidden_size)

  def forward(self, arranger_id, mel_db, token):
    return torch.cat([self.arranger_embeddings(arranger_id), mel_db, self.token_embeddings(token)], dim=-2)

In [ ]:
class MusicDataset(Dataset):
    def __init__(self, tokenizer, embbeding, max_output_length=4096, music_folder="song2midi/dataset/song", midi_folder="song2midi/dataset/midi", compressor=None):
        self.tokenizer = tokenizer
        self.embbeding = embbeding
        self.max_output_length = max_output_length
        self.music_folder = music_folder
        self.midi_folder = midi_folder
        
        self.music_ids = [name.split(".")[0] for name in os.listdir(music_folder)]
        
        uncheck_midis = [name.split(".")[:2] for name in os.listdir(midi_folder)]
        self.midis = [midi for midi in uncheck_midis if midi[0] in self.music_ids] # [music_id, arranger_id]

    def __len__(self):
        return len(self.music_ids)
    
    def __getitem__(self, idx):
        music_id, arranger_id = self.midis[idx]

        # music
        music_path = os.path.join(self.music_folder, f"{music_id}.mp3")
        waveform, sr = torchaudio.load(music_path)

        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        mel_transform = torchaudio.transforms.MelSpectrogram(sample_rate=sr)
        mel = mel_transform(waveform)
        mel_db = torchaudio.transforms.AmplitudeToDB()(mel)

        mel_scaled = torch.nn.functional.interpolate(mel_db.unsqueeze(1), size=(128, 1023), mode='bilinear', align_corners=False).squeeze(1)
        mel_shape = mel_scaled.shape
        mel_scaled = mel_scaled.reshape(mel_shape[0], mel_shape[2], mel_shape[1])

        input_embed = self.embbeding(torch.tensor([[int(arranger_id)]]), mel_scaled)
        attention_mask = torch.ones(input_embed.shape[:2], dtype=torch.int32)

        global_attention_mask = torch.zeros(input_embed.shape[:2], dtype=torch.int32)
        global_attention_mask[:, 0] = 1

        # midi
        midi_path = os.path.join(self.midi_folder, f"{music_id}.{arranger_id}.mid")
        score = Score(midi_path)

        miditok.utils.merge_same_program_tracks(score.tracks)

        midi = torch.tensor(self.tokenizer.encode(score), dtype=torch.int32)
        # print(torch.tensor([self.tokenizer["BOS_None"]]).shape, midi.shape, torch.tensor([self.tokenizer["EOS_None"]]).shape)
        midi = torch.cat([torch.tensor([[self.tokenizer["BOS_None"]]]), midi, torch.tensor([[self.tokenizer["EOS_None"]]])], dim=-1)

        # padding with -100
        if midi.shape[0] < self.max_output_length:
            midi = torch.cat([midi, torch.ones((midi.shape[0], self.max_output_length - midi.shape[1]), dtype=torch.int32) * -100], dim=-1)
        else:
            midi = midi[:, :self.max_output_length]

        return {"inputs_embeds": input_embed.squeeze(0), "attention_mask": attention_mask.squeeze(0), "global_attention_mask": global_attention_mask.squeeze(0), "labels": midi.squeeze(0)}


In [62]:
def preprocess_data(tokenizer, sample_rate=22050, music_folder="song2midi/dataset/song", midi_folder="song2midi/dataset/midi", output_folder="song2midi/dataset/preprocess"):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if not os.path.exists(os.path.join(output_folder, "mel")):
        os.makedirs(os.path.join(output_folder, "mel"))

    if not os.path.exists(os.path.join(output_folder, "midi")):
        os.makedirs(os.path.join(output_folder, "midi"))
                    
    music_ids = [name.split(".")[0] for name in os.listdir(music_folder)]
    uncheck_midis = [name.split(".")[:2] for name in os.listdir(midi_folder)]
    midis = [midi for midi in uncheck_midis if midi[0] in music_ids] # [music_id, arranger_id]

    skipped_music = []

    for music_id in music_ids:
        music_path = os.path.join(music_folder, f"{music_id}.mp3")
        waveform, sr = librosa.load(music_path, sr=sample_rate)

        mel_transform = torchaudio.transforms.MelSpectrogram(sample_rate=sr, n_fft=2048, hop_length=1536, n_mels=128)
        mel = mel_transform(torch.tensor(waveform).unsqueeze(0))
        mel_db = torchaudio.transforms.AmplitudeToDB()(mel)

        mel_shape = mel_db.shape
        if mel_shape[2] > 4096: # have to skip this music
            print(f"music id: {music_id} is too long skipping")
            skipped_music.append(music_id)
            continue

        mel_db = mel_db.reshape(mel_shape[0], mel_shape[2], mel_shape[1])

        np.save(os.path.join(output_folder, "mel", f"{music_id}.mel.npy"), mel_db.numpy())

    for music_id, arranger_id in midis:
        if music_id in skipped_music:
            continue
        
        midi_path = os.path.join(midi_folder, f"{music_id}.{arranger_id}.mid")
        score = Score(midi_path)
        miditok.utils.merge_same_program_tracks(score.tracks)
        midi_encoded = tokenizer.encode(score)
        np.save(os.path.join(output_folder, "midi", f"{music_id}.{arranger_id}.midi.npy"), midi_encoded)


In [5]:
def mel_interpolate(mel):
    mel_shape = mel.shape[1]
    return nn.functional.interpolate(mel.unsqueeze(1), size=(math.floor(mel_shape / 4), 128), mode='bilinear', align_corners=False).squeeze(1)

In [66]:
class MusicDataset(Dataset):
    def __init__(self, tokenizer, embbeding, max_output_length=4096, max_input_length=1024, mel_processing=None, preprocess_folder="song2midi/dataset/preprocess"):
        self.tokenizer = tokenizer
        self.embbeding = embbeding
        self.max_output_length = max_output_length
        self.max_input_length = max_input_length
        self.preprocess_folder = preprocess_folder
        self.mel_processing = mel_processing
        
        self.data_files = [name.split(".")[:2] for name in os.listdir(os.path.join(preprocess_folder, "midi"))]

    def __len__(self):
        return len(self.data_files)
    
    def __getitem__(self, idx):
        music_id, arranger_id = self.data_files[idx]

        # load precomputed mel'
        mel_db = np.load(os.path.join(self.preprocess_folder, "mel", f"{music_id}.mel.npy"))
        mel_db = torch.tensor(mel_db)

        if self.mel_processing:
            mel_db = self.mel_processing(mel_db)

        if mel_db.shape[1] < self.max_input_length - 1:
            num_pad = self.max_input_length - mel_db.shape[1] - 1
            mel_padded = torch.cat([mel_db, torch.zeros((1, num_pad, mel_db.shape[2]))], dim=1)
        else:
            num_pad = 1
            mel_padded = mel_db[:, :self.max_input_length - 1]

        input_embed = self.embbeding(torch.tensor([[int(arranger_id)]]), mel_padded)
        attention_mask = torch.cat([torch.ones(mel_db.shape[:2], dtype=torch.int32), torch.zeros((mel_db.shape[0], num_pad + 1))], dim=1)

        global_attention_mask = torch.zeros(input_embed.shape[:2], dtype=torch.int32)
        global_attention_mask[:, 0] = 1

        # load precomputed midi
        midi_encoded = np.load(os.path.join(self.preprocess_folder, "midi", f"{music_id}.{arranger_id}.midi.npy"))
        midi = torch.tensor(midi_encoded, dtype=torch.int32)
        midi = torch.cat([torch.tensor([[self.tokenizer["BOS_None"]]]), midi, torch.tensor([[self.tokenizer["EOS_None"]]])], dim=-1)

        # padding with -100
        if midi.shape[0] < self.max_output_length:
            midi = torch.cat([midi, torch.ones((midi.shape[0], self.max_output_length - midi.shape[1]), dtype=torch.int32) * -100], dim=-1)
        else:
            midi = midi[:, :self.max_output_length]

        return {"inputs_embeds": input_embed.squeeze(0), 
                "attention_mask": attention_mask.squeeze(0), 
                #"global_attention_mask": global_attention_mask.squeeze(0), 
                "labels": midi.squeeze(0)}


In [54]:
class MusicDataset(Dataset):
    def __init__(self, tokenizer, embbeding, max_output_length=4096, max_input_length=4096, mel_processing=None, preprocess_folder="song2midi/dataset/preprocess"):
        self.tokenizer = tokenizer
        self.embbeding = embbeding
        self.max_output_length = max_output_length
        self.max_input_length = max_input_length
        self.preprocess_folder = preprocess_folder
        self.mel_processing = mel_processing
        
        self.data_files = [name.split(".")[:2] for name in os.listdir(os.path.join(preprocess_folder, "midi"))]

    def __len__(self):
        return len(self.data_files)
    
    def __getitem__(self, idx):
        music_id, arranger_id = self.data_files[idx]

        # load precomputed mel'
        mel_db = np.load(os.path.join(self.preprocess_folder, "mel", f"{music_id}.mel.npy"))
        mel_db = torch.tensor(mel_db)

        if self.mel_processing:
            mel_db = self.mel_processing(mel_db)

        if mel_db.shape[1] > self.max_input_length - 1:
            mel_db = mel_db[:, :self.max_input_length - 1]

        pad_mel_db = nn.functional.pad(mel_db, (0, 0, 0, self.max_input_length - mel_db.shape[1]), value=0)

        midi_encoded = np.load(os.path.join(self.preprocess_folder, "midi", f"{music_id}.{arranger_id}.midi.npy"))
        midi = torch.tensor(midi_encoded, dtype=torch.int32)
        midi = torch.cat([torch.tensor([[self.tokenizer["BOS_None"]]]), midi, torch.tensor([[self.tokenizer["EOS_None"]]])], dim=-1)

        pad_midi = nn.functional.pad(midi, (0, self.max_output_length - midi.shape[1]), value=self.tokenizer["PAD_None"])

        pad_midi_labels = nn.functional.pad(midi, (0, self.max_output_length - midi.shape[1]), value=-100)

        # print(pad_mel_db)
        # print(pad_midi.shape)
        # print(pad_midi_labels.shape)

        input_embed = self.embbeding(torch.tensor([[int(arranger_id)]]), pad_mel_db, pad_midi)

        labels = torch.cat([torch.ones((1, 1), dtype=torch.int32) * -100, torch.ones((pad_mel_db.shape[:2]), dtype=torch.int32) * -100, pad_midi_labels], dim=-1)


        return {"inputs_embeds": input_embed.squeeze(0), 
                "labels": labels.squeeze(0)}


In [6]:
TOKENIZER_PARAMS = {
    "pitch_range": (0, 127),
    "beat_res": {(0, 4): 8, (4, 12): 4},
    "num_velocities": 32,
    "special_tokens": ["PAD", "BOS", "EOS", "MASK"],
    "use_chords": True,
    "use_rests": False,
    "use_tempos": True,
    "use_time_signatures": False,
    "use_programs": False,
    "num_tempos": 32,  # number of tempo bins
    "tempo_range": (40, 250),  # (min, max)
}
remi_config = TokenizerConfig(**TOKENIZER_PARAMS)

/tmp/ipykernel_126903/2731034332.py:14: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  remi_config = TokenizerConfig(**TOKENIZER_PARAMS)


In [7]:
tokenizer = REMI(remi_config)

In [15]:
tokenizer = REMI(params="song2midi/dataset/tokenizer.json")

/home/bk/miniconda3/envs/keras/lib/python3.11/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/home/bk/miniconda3/envs/keras/lib/python3.11/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)


In [8]:
midi_paths = list(Path("song2midi", "dataset", "midi").glob("*.mid"))

tokenizer.train(vocab_size=50265, files_paths=midi_paths)

In [11]:
tokenizer.save_params("song2midi/dataset/tokenizer.json")

In [65]:
preprocess_data(tokenizer)

In [63]:
preprocess_data(tokenizer, music_folder="song2midi/dataset/song_eval", midi_folder="song2midi/dataset/midi_eval", output_folder="song2midi/dataset/preprocess_eval")

In [55]:
dataset = MusicDataset(tokenizer, MambaEmbedding(tokenizer.vocab_size), max_output_length=4096, max_input_length=4096, mel_processing=None)

In [68]:
eval_dataset = MusicDataset(tokenizer, MambaEmbedding(tokenizer.vocab_size), max_output_length=4096, max_input_length=4096, mel_processing=None, preprocess_folder="song2midi/dataset/preprocess_eval")

In [53]:
for a in dataset:
    print(a["inputs_embeds"].shape, a["labels"].shape)
    break

tensor([[[-100.0000,  -57.9684,  -56.6364,  ...,  -12.9613,  -12.0673,
           -10.0208],
         [ -19.8567,  -12.8137,  -16.1480,  ...,   -4.3473,   -1.0316,
            -0.6634],
         [  -4.0322,   -0.6607,    2.2929,  ...,   -7.8962,   -9.6513,
            -4.9240],
         ...,
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000]]])
torch.Size([1, 4096])
torch.Size([1, 4096])
torch.Size([8193, 128]) torch.Size([8193])


In [32]:
mamba_config = MambaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=128,
    pad_token_id=tokenizer["PAD_None"],
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)

model = MambaForCausalLM(mamba_config).cuda()

In [12]:
outp = model(inputs_embeds=torch.rand((1, 4096, 128)).cuda(), labels=torch.randint(0, 50265, (1, 4096)).cuda())

In [33]:
outp = model(inputs_embeds=dataset[0]["inputs_embeds"].unsqueeze(0).cuda(), labels=dataset[0]["labels"].unsqueeze(0).cuda())

In [36]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

In [56]:
dataloader_train = DataLoader(dataset, batch_size=3, shuffle=True)
# dataloader_eval = DataLoader(eval_dataset, batch_size=3, shuffle=False)

num_epochs = 10

optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=(len(dataloader_train) * num_epochs))

for epoch in range(num_epochs):

    # training
    train_loss = 0
    model.train()
    for batch in tqdm(dataloader_train):
        optimizer.zero_grad()
        outp = model(inputs_embeds=batch["inputs_embeds"].cuda(), labels=batch["labels"].cuda())
        loss = outp.loss
        loss.backward()

        train_loss += loss.item()
        optimizer.step()
        scheduler.step()

    avg_train_loss = train_loss / len(dataloader_train)
    print(f"epoch {epoch} train loss: {avg_train_loss}")

    # # evaluation
    # eval_loss = 0
    # model.eval()
    # for batch in tqdm(dataloader_eval):
    #     with torch.no_grad():
    #         outp = model(inputs_embeds=batch["inputs_embeds"].cuda(), labels=batch["labels"].cuda())
    #         loss = outp.loss

    #         eval_loss += loss.item()

    # avg_eval_loss = eval_loss / len(dataloader_eval)
    # print(f"epoch {epoch} eval loss: {avg_eval_loss}")



  0%|                                                                            | 0/15 [00:00<?, ?it/s]

In [49]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

In [69]:
train_dataloader = DataLoader(dataset, batch_size=5, shuffle=True)
val_dataloader = DataLoader(eval_dataset, batch_size=1)

optimizer = AdamW(model.parameters(), lr=0.0001)
num_training_steps = len(train_dataloader) * 500
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Training Loop
num_train_epochs = 500
for epoch in range(num_train_epochs):
    # Training
    model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}/{num_train_epochs}"):
        inputs_embeds = batch['inputs_embeds'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        
        outputs = model(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs["loss"]
        loss.backward()
        
        train_loss += loss.item()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Average Training Loss: {avg_train_loss}")

    if (epoch + 1) % 2 == 0:
        # Evaluation
        model.eval()
        eval_loss = 0
        eval_steps = 0
        
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc=f"Evaluating Epoch {epoch+1}/{num_train_epochs}"):
                inputs_embeds = batch['inputs_embeds'].to('cuda')
                attention_mask = batch['attention_mask'].to('cuda')
                labels = batch['labels'].to('cuda')
                
                outputs = model(
                    inputs_embeds=inputs_embeds,
                    attention_mask=attention_mask,
                    labels=labels
                )
                
                loss = outputs["loss"]
                eval_loss += loss.item()
                
                eval_steps += 1
        
        avg_eval_loss = eval_loss / eval_steps
        print(f"Average Evaluation Loss: {avg_eval_loss}")

/home/bk/miniconda3/envs/keras/lib/python3.11/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training Epoch 1/500: 100%|███████████████████████████████████████████████████| 9/9 [00:35<00:00,  3.97s/it]


Average Training Loss: 10.99042797088623


Training Epoch 2/500: 100%|███████████████████████████████████████████████████| 9/9 [00:35<00:00,  3.94s/it]


Average Training Loss: 10.955936643812391


Evaluating Epoch 2/500: 100%|█████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.27it/s]


...

Average Training Loss: 10.043963326348198


Evaluating Epoch 12/500: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.28it/s]


Average Evaluation Loss: 9.93946647644043


Training Epoch 13/500:  22%|███████████                                       | 2/9 [00:13<00:47,  6.83s/it]


KeyboardInterrupt: 

In [35]:
out

LEDSeq2SeqLMOutput(loss=tensor(10.8519, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[ 0.1528,  0.0000,  1.7232,  ...,  0.0340, -0.2018,  0.2016],
         [ 0.0276,  0.0000,  0.0030,  ..., -0.0170,  0.3117, -0.0377],
         [ 0.1439,  0.0000, -0.2117,  ..., -0.1489, -0.0806,  0.2360],
         ...,
         [-0.0684,  0.0000,  0.4367,  ..., -0.0780,  0.1614, -0.0231],
         [-0.1011,  0.0000,  0.0282,  ...,  0.4681,  0.0164,  0.3435],
         [-0.4156,  0.0000, -0.0677,  ...,  0.2259, -0.0516, -0.0998]]],
       device='cuda:0', grad_fn=<AddBackward0>), past_key_values=None, decoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, encoder_last_hidden_state=tensor([[[ 0.0419,  0.3264, -0.3593,  ...,  0.9347, -0.8379, -1.3963],
         [-0.3056,  0.7967, -0.5429,  ...,  1.9927,  0.8124,  0.7674],
         [-0.5460,  0.8657, -1.1845,  ..., -1.8351,  0.3854,  0.0470],
         ...,
         [ 2.3461,  2.7531,  2.2392,  ..., -0.1812, -0.4449, -0.2

In [31]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

2024-05-28 18:55:23.024028: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-28 18:55:23.093224: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 18:55:23.710833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [32]:
training_args = Seq2SeqTrainingArguments(
    output_dir="song2midi/results",
    #auto_find_batch_size=True,
    per_device_train_batch_size=5,
    log_level="passive",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    save_safetensors=False,
    num_train_epochs=500,
    learning_rate=0.0001,
)

In [33]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset,
)

In [34]:
log = trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
10,10.994000
20,10.928000
30,10.859600
40,10.746300
50,10.598700
60,10.432600
70,10.318100
80,10.144100
90,10.005300
100,9.897300


KeyboardInterrupt: 

In [16]:
out.keys()

odict_keys(['logits', 'past_key_values', 'encoder_last_hidden_state'])

In [ ]:
miditok.utils.filter_dataset(Path("song2midi", "dataset", "midi").glob("*.mid"), )

In [55]:
from symusic import Score

In [71]:
aaa = Score("song2midi/dataset/midi/This_Game.mid")

In [77]:
miditok.utils.merge_same_program_tracks(aaa.tracks)

In [78]:
aaa

Score(ttype=Tick, tpq=480, begin=0, end=138155, tracks=1, notes=1488, time_sig=1, key_sig=4, markers=0, lyrics=0)

In [16]:
y, sr = librosa.load("song2midi/dataset/wav/Idol.wav")

In [17]:
y.shape, sr

((4978177,), 22050)

In [19]:
mel = librosa.feature.melspectrogram(y=y, sr=sr)

In [21]:
mel.shape

(128, 9724)

In [31]:
4978177 - 5418423

-440246